In [ ]:
# @title Default title text
!pip install kaggle

In [ ]:
# Upload kaggle.json
from google.colab import files
files.upload()  # This will prompt you to upload the kaggle.json file

# Create the .kaggle directory
import os
os.makedirs('/root/.kaggle', exist_ok=True)

# Move the kaggle.json file to the .kaggle directory
import shutil
shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d adityajn105/flickr8k

# Unzip the dataset
!unzip flickr8k.zip

In [ ]:
pip install textblob


In [ ]:
pip install nltk

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Reshape, Embedding, concatenate, LSTM, Dropout, add
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import seaborn as sns
from textwrap import wrap
from textblob import TextBlob
import warnings

In [ ]:
plt.rcParams['font.size'] = 12
sns.set_style("dark")
warnings.filterwarnings('ignore')

In [ ]:
image_path = '/content/Images'

In [ ]:
data = pd.read_csv("/content/captions.txt")
data.head()

In [ ]:
def readImage(path, img_size=224):
    img = load_img(path, color_mode='rgb', target_size=(img_size, img_size))
    img = img_to_array(img)
    img = img / 255.
    return img

# Function to display images with captions
def display_images(temp_df):
    temp_df = temp_df.reset_index(drop=True)
    plt.figure(figsize=(20, 20))
    n = 0
    for i in range(15):
        n += 1
        plt.subplot(5, 5, n)
        plt.subplots_adjust(hspace=0.7, wspace=0.3)
        image = readImage(f"/content/Images/{temp_df.image[i]}")
        plt.imshow(image)
        plt.title("\n".join(wrap(temp_df.caption[i], 20)))
        plt.axis("off")
    plt.show()


In [ ]:
def text_preprocessing(data):
    data['caption'] = data['caption'].apply(lambda x: x.lower())
    data['caption'] = data['caption'].apply(lambda x: x.replace("[^A-Za-z]", ""))
    data['caption'] = data['caption'].apply(lambda x: x.replace("\s+", " "))
    data['caption'] = data['caption'].apply(lambda x: " ".join([word for word in x.split() if len(word) > 1]))
    data['caption'] = "startseq " + data['caption'] + " endseq"
    return data

data = text_preprocessing(data)
captions = data['caption'].tolist()

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions)
vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(caption.split()) for caption in captions)

In [ ]:
# Splitting data
images = data['image'].unique().tolist()
nimages = len(images)
split_index = round(0.85 * nimages)
train_images = images[:split_index]
val_images = images[split_index:]

train = data[data['image'].isin(train_images)]
test = data[data['image'].isin(val_images)]
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

In [ ]:
model = DenseNet201()
fe = Model(inputs=model.input, outputs=model.layers[-2].output)

img_size = 224
features = {}
for image in tqdm(data['image'].unique().tolist()):
    img = load_img(os.path.join(image_path,image),target_size=(img_size,img_size))
    img = img_to_array(img)
    img = img/255.
    img = np.expand_dims(img,axis=0)
    feature = fe.predict(img, verbose=0)
    features[image] = feature

82524592/82524592 [==============================] - 2s 0us/step


100%|██████████| 8091/8091 [45:48<00:00,  2.94it/s]


In [ ]:
# Custom data generator
class CustomDataGenerator(Sequence):
    def __init__(self, df, X_col, y_col, batch_size, directory, tokenizer,
                 vocab_size, max_length, features, shuffle=True):
        self.df = df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.directory = directory
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.features = features
        self.shuffle = shuffle
        self.n = len(self.df)

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

    def __len__(self):
        return self.n // self.batch_size

    def __getitem__(self, index):
        batch = self.df.iloc[index * self.batch_size:(index + 1) * self.batch_size, :]
        X1, X2, y = self.__get_data(batch)
        return (X1, X2), y

    def __get_data(self, batch):
        X1, X2, y = list(), list(), list()
        images = batch[self.X_col].tolist()
        for image in images:
            feature = self.features[image][0]
            captions = batch.loc[batch[self.X_col] == image, self.y_col].tolist()
            for caption in captions:
                seq = self.tokenizer.texts_to_sequences([caption])[0]
                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]
                    in_seq = pad_sequences([in_seq], maxlen=self.max_length)[0]
                    out_seq = to_categorical([out_seq], num_classes=self.vocab_size)[0]
                    X1.append(feature)
                    X2.append(in_seq)
                    y.append(out_seq)
        X1, X2, y = np.array(X1), np.array(X2), np.array(y)
        return X1, X2, y

In [ ]:
# Model definition
input1 = Input(shape=(1920,))
input2 = Input(shape=(max_length,))
img_features = Dense(256, activation='relu')(input1)
img_features_reshaped = Reshape((1, 256), input_shape=(256,))(img_features)
sentence_features = Embedding(vocab_size, 256, mask_zero=False)(input2)
merged = concatenate([img_features_reshaped, sentence_features], axis=1)
sentence_features = LSTM(256)(merged)
x = Dropout(0.5)(sentence_features)
x = add([x, img_features])
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(vocab_size, activation='softmax')(x)
caption_model = Model(inputs=[input1, input2], outputs=output)
caption_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# Plot model
from tensorflow.keras.utils import plot_model
plot_model(caption_model)
caption_model.summary()


In [ ]:
# Generators
train_generator = CustomDataGenerator(df=train, X_col='image', y_col='caption', batch_size=64, directory=image_path,
                                      tokenizer=tokenizer, vocab_size=vocab_size, max_length=max_length, features=features)
validation_generator = CustomDataGenerator(df=test, X_col='image', y_col='caption', batch_size=64, directory=image_path,
                                           tokenizer=tokenizer, vocab_size=vocab_size, max_length=max_length, features=features)

In [ ]:

# Callbacks
model_name = "model.h5"
checkpoint = ModelCheckpoint(model_name, monitor="val_loss", mode="min", save_best_only=True, verbose=1)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, restore_best_weights=True)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.2, min_lr=1e-8)

In [ ]:
# Training the model
history = caption_model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[checkpoint, earlystopping, learning_rate_reduction]
)


In [ ]:
from nltk.translate.bleu_score import corpus_bleu

# Initialize references dictionary
references = {}

# Batch size for processing
batch_size = 100

# Initialize lists for storing predictions and references
predicted_captions = []

# Process batches of data
for start_idx in range(0, len(test), batch_size):
    end_idx = min(start_idx + batch_size, len(test))
    batch = test.iloc[start_idx:end_idx]

    # Generate predictions for the batch
    batch_predictions = []
    for image in batch['image']:
        predicted_caption = predict_caption(caption_model, image, tokenizer, max_length, features)
        batch_predictions.append(predicted_caption)
    predicted_captions.extend(batch_predictions)

    # Prepare reference captions for the batch
    for index, row in batch.iterrows():
        img_name = row['image']
        if img_name not in references:
            references[img_name] = []
        references[img_name].append(row['caption'].split())

# Convert references to the format expected by nltk corpus_bleu
reference_bleu = [[captions] for _, captions in references.items()]
predicted_bleu = [caption.split() for caption in predicted_captions]

# Calculate BLEU score
bleu_score = corpus_bleu(reference_bleu, predicted_bleu)
print(f"BLEU Score: {bleu_score}")


In [ ]:
# Plotting the training history
plt.figure(figsize=(20, 8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# Function to map indices to words
def idx_to_word(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [ ]:
# Function to predict caption
def predict_caption(model, image, tokenizer, max_length, features):
    feature = features[image]
    in_text = "startseq"
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], max_length)
        y_pred = model.predict([feature, sequence])
        y_pred = np.argmax(y_pred)
        word = idx_to_word(y_pred, tokenizer)
        if word is None:
            break
        in_text += " " + word
        if word == 'endseq':
            break
    return in_text

In [ ]:
# Performing sentiment analysis on captions
def analyze_sentiment(caption):
    analysis = TextBlob(caption)
    return analysis.sentiment.polarity, analysis.sentiment.subjectivity

In [ ]:

# Generating and displaying captions with sentiment analysis
samples = test.sample(15)
samples.reset_index(drop=True, inplace=True)
for index, record in samples.iterrows():
    img = load_img(os.path.join(image_path, record['image']), target_size=(224, 224))
    img = img_to_array(img)
    img = img / 255.
    caption = predict_caption(caption_model, record['image'], tokenizer, max_length, features)
    samples.loc[index, 'caption'] = caption
    polarity, subjectivity = analyze_sentiment(caption)
    samples.loc[index, 'polarity'] = polarity
    samples.loc[index, 'subjectivity'] = subjectivity

In [ ]:
def display_images_with_sentiment(temp_df, image_path):
    temp_df = temp_df.reset_index(drop=True)
    plt.figure(figsize=(20, 20))
    n = 0
    for i in range(len(temp_df)):
        n += 1
        plt.subplot(5, 5, n)
        plt.subplots_adjust(hspace=0.7, wspace=0.3)
        image_path_full = os.path.join(image_path, temp_df.image[i])
        image = readImage(image_path_full)
        plt.imshow(image)

        caption = clean_caption(temp_df.caption[i])
        sentiment = TextBlob(caption).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity

        # Print caption and sentiment analysis results for each image
        print(f"Caption: {caption}")
        print(f"Polarity: {polarity:.2f}, Subjectivity: {subjectivity:.2f}")

        # Determine sentiment label based on polarity
        if polarity > 0:
            sentiment_label = "Positive"
        elif polarity < 0:
            sentiment_label = "Negative"
        else:
            sentiment_label = "Neutral"

        print(f"Sentiment: {sentiment_label}\n")

        title = f"{caption}\nPolarity: {polarity:.2f}, Subjectivity: {subjectivity:.2f}"
        plt.title("\n".join(wrap(title, 40)))
        plt.axis("off")

    plt.tight_layout(pad=2.0)

# Example usage:
display_images_with_sentiment(samples, '/content/Images')
